# Book Recommender System Using BX-Book-Ratings Dataset

## Project Introduction

Welcome to our machine learning-based book recommender system project. The aim of this Jupyter notebook is to guide you through the process of designing, implementing, and evaluating a recommender system that suggests books to users based on their historical preferences and ratings. 

Using the BX-Book-Ratings dataset, which comprises user ratings for a vast array of books, we will explore various recommendation algorithms, including collaborative filtering, content-based filtering, and hybrid methods, to identify the most effective approach for our system.

## Objectives

- **Data Exploration**: Understand the structure and characteristics of the BX-Book-Ratings dataset.
- **Preprocessing**: Clean and preprocess the data to ensure it is suitable for analysis and modeling.
- **Model Development**: Implement various recommendation algorithms and evaluate their performance.
- **Recommendation**: Provide personalized book recommendations to users.

## Dataset Overview

The BX-Book-Ratings dataset is a rich collection of user ratings for books, making it an ideal resource for developing a recommender system. It includes:

- User IDs: Unique identifiers for each user who has rated books.
- ISBN: Standard book numbers which uniquely identify books.
- Book Ratings: Ratings given by users to books on a scale.

This dataset offers a comprehensive foundation for understanding user preferences and predicting potential book recommendations that align with their interests.



In [31]:
import pandas as pd

# Adjusting the call to pd.read_csv() to use on_bad_lines parameter
books = pd.read_csv('data/BX-Books.csv', sep=";", on_bad_lines='skip', encoding='latin-1')

# Display the first few rows of the dataframe to verify successful loading
print(books.head())


         ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

            Book-Author Year-Of-Publication                   Publisher  \
0    Mark P. O. Morford                2002     Oxford University Press   
1  Richard Bruce Wright                2001       HarperFlamingo Canada   
2          Carlo D'Este                1991             HarperPerennial   
3      Gina Bari Kolata                1999        Farrar Straus Giroux   
4       E. J. W. Barber                1999  W. W. Norton &amp; Company   

                                         Image-URL-S  \
0  http://images.amazon.com/images/P/0195153448.0...   
1  http://images.amazon.com/

C:\Users\User\AppData\Local\Temp\ipykernel_39132\1106502186.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('data/BX-Books.csv', sep=";", on_bad_lines='skip', encoding='latin-1')


In [32]:
# Retrieve the large image URL for the book at the 237th index position
large_image_url = books.iloc[10]['Image-URL-L']

print(large_image_url)


http://images.amazon.com/images/P/0771074670.01.LZZZZZZZ.jpg


This code snippet will update your books DataFrame to only include the columns: 'ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', and 'Image-URL-L'. It then displays the first few rows of the updated DataFrame to ensure the changes have been applied correctly. By focusing on these columns, you're preparing your dataset to support both the functionality of your book recommender system and the visual presentation of book recommendations.

In [33]:
# Narrowing down the dataset to essential columns
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-L']]

# Display the first few rows of the modified dataframe to verify the changes
books.head()


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


## Loading User Data

Now, we proceed to load the user dataset, which contains information about the users who have provided book ratings. This dataset is crucial for understanding user demographics and enabling personalized book recommendations. The dataset is stored in `BX-Users.csv` and will be loaded into a DataFrame for further analysis. We'll ensure to handle any malformed lines by skipping them and decode the file using 'latin-1' encoding to properly handle special characters found in the dataset.


In [34]:

# Load the BX-Users.csv file into a DataFrame, adjusting for the deprecated parameter
users = pd.read_csv('data/BX-Users.csv', sep=";", on_bad_lines='skip', encoding='latin-1')

# Display the first few rows of the users dataframe to verify successful loading
users.head()


,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


#### Renaming Columns for Clarity

In our `users` DataFrame, some column names are not consistent with the conventional Python naming conventions or might be considered unclear for some readers. To enhance readability and maintain consistency throughout our analysis, we will rename these columns:

- `User-ID` to `user_id`: This makes the column name lowercase and follows the Python convention of using underscores to separate words.
- `Location` to `location`: Although the original name is clear, we lowercase it for consistency with other column names.
- `Age` to `age`: We lowercase the column name for consistency.

This renaming process ensures our DataFrame columns have descriptive, consistent names that follow common Python conventions.


In [35]:
users.rename(columns={"User-ID":'user_id',
                      'Location':'location',
                     "Age":'age'},inplace=True)

## Loading Book Ratings Data

Next, we load the book ratings dataset from the `BX-Book-Ratings.csv` file. This dataset is pivotal for our recommendation system as it contains the ratings users have given to various books. Each entry includes a user identifier, an ISBN number for the book, and the rating given by the user. To ensure a smooth loading process, we handle any malformed lines by skipping them and use 'latin-1' encoding to accurately interpret special characters within the dataset.


In [36]:
# Load the BX-Book-Ratings.csv file into a DataFrame, adjusting for the updated parameter
ratings = pd.read_csv('data/BX-Book-Ratings.csv', sep=";", on_bad_lines='skip', encoding='latin-1')

# Display the first few rows of the ratings dataframe to verify successful loading
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


#### Renaming Columns in Ratings DataFrame

To ensure our `ratings` DataFrame aligns with the naming conventions used throughout our analysis and to improve the readability of our dataset, we have decided to rename certain columns:

- `User-ID` to `user_id`: Standardizing the column name to lowercase with underscores for better readability and consistency with Python variable naming conventions.
- `Book-Rating` to `rating`: Simplifying the column name while retaining its clear meaning, and aligning it with the lowercase naming convention.

These changes help in maintaining a uniform naming convention across all dataframes in our project, making the data easier to work with and understand.


In [37]:
ratings.rename(columns={"User-ID":'user_id',
                      'Book-Rating':'rating'},inplace=True)

In [38]:
print(books.shape, users.shape, ratings.shape, sep='\n')


(271360, 6)
(278858, 3)
(1149780, 3)


### Filtering Users Based on Activity Level

In our dataset, we aim to focus on highly active users to ensure our recommendations are based on users with a significant history of ratings. To achieve this, we identify users who have rated more than 200 books. This threshold helps us filter out casual users and concentrate on those who are likely to provide more reliable and diverse ratings data.

After identifying these active users, we filter our `ratings` DataFrame to only include the ratings from users meeting this criterion. This step ensures our analysis and subsequent recommendation model are built upon a robust and engaged user base.


In [42]:
# Identifying users who have rated more than 200 books
user_counts = ratings['user_id'].value_counts()
active_users = user_counts[user_counts > 200].index.tolist()

# Filtering the ratings DataFrame to only include active users
filtered_ratings = ratings[ratings['user_id'].isin(active_users)]


filtered_ratings.head()


,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


## Merging Ratings with Book Details

To enrich our ratings data with detailed book information, we perform a merge operation between the `ratings` and `books` DataFrames. This merge is based on the 'ISBN' column, which is common to both datasets. The resulting DataFrame, `ratings_with_books`, combines user ratings with corresponding book details, such as title, author, and publication year.

This enhanced dataset provides a comprehensive view of each rating, allowing us to understand not just the user's rating but also which book it pertains to. It serves as a foundational dataset for further analysis, enabling more nuanced insights into user preferences and reading habits.


In [50]:
# Perform the merge operation
ratings_with_books = filtered_ratings.merge(books, on='ISBN')

# Display the first few rows of the merged DataFrame to verify the merge
ratings_with_books.head()


,user_id,ISBN,rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...


In [51]:
# Calculate the number of ratings for each book title
number_rating = ratings_with_books.groupby('Book-Title')['rating'].count().reset_index()
number_rating.rename(columns={'rating':'num_of_rating'},inplace=True)

number_rating.head()


,Book-Title,num_of_rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [52]:
# Assuming 'Book-Title' is the correct column name for titles in both DataFrames
final_rating = ratings_with_books.merge(number_rating, on='Book-Title')
final_rating.head()



,user_id,ISBN,rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


In [54]:
# Filter the final_rating DataFrame to include only books with at least 50 ratings
final_rating = final_rating[final_rating['num_of_rating'] >= 50]
final_rating.head()


,user_id,ISBN,rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


In [ ]:
# Drop duplicate entries based on 'user_id' and 'title'
final_rating.drop_duplicates(['user_id', 'Book-Title'], inplace=True)


In [ ]:
final_rating.shape


(59850, 9)

## Creating a Pivot Table for Collaborative Filtering

To advance our book recommendation system, we construct a pivot table from the `final_rating` DataFrame. This pivot table reorganizes our data, setting book titles as the index (rows), user IDs as the columns, and the individual ratings as the values within the table. This structure is instrumental for collaborative filtering techniques, as it allows us to easily identify and compare the ratings across different users for the same books.

### Purpose of the Pivot Table

The pivot table serves several key purposes in the context of recommendation systems:

- **Facilitates Similarity Computation**: By aligning users and books in a matrix form, we can efficiently compute similarities between items (books) or between users, which is the cornerstone of collaborative filtering approaches.
- **Prepares for Machine Learning**: The structured format makes it straightforward to apply various machine learning algorithms directly to the data, especially those designed for recommendation tasks, such as matrix factorization.
- **Identifies Missing Values**: The pivot table clearly indicates where ratings are missing (represented as NaNs), highlighting potential areas for imputation or special handling depending on the chosen algorithm.

### Handling Sparse Data

A common challenge with pivot tables in recommendation systems is their sparsity, as not all users rate all books. Strategies to address this include:

- **Filling Missing Values**: Depending on the algorithm, missing values can be filled with zeros, the mean rating, or through more sophisticated imputation techniques.
- **Sparse Matrix Representation**: For larger datasets, using sparse matrices helps manage memory efficiently, retaining only the non-zero elements in the table.

The creation of this pivot table marks a critical step in our journey to develop a nuanced and effective book recommendation system, laying the groundwork for the application of collaborative filtering and other recommendation techniques.


In [55]:
# Create a pivot table with book titles as rows, user IDs as columns, and ratings as values
book_pivot = final_rating.pivot_table(columns='user_id', index='Book-Title', values='rating')


In [56]:
book_pivot


user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [57]:
book_pivot.shape


(742, 888)

In [58]:
book_pivot.fillna(0, inplace=True)

## Training the Recommendation Model

With our data prepared in a pivot table format, the next step in building our book recommendation system is to train a model capable of finding similar books based on user ratings. To efficiently handle the data and prepare it for model training, we convert our pivot table into a sparse matrix. This transformation is crucial for managing memory and computational efficiency, given the sparsity of our data.

### Converting to Sparse Matrix

We use the `csr_matrix` from SciPy to convert our pivot table into a Compressed Sparse Row (CSR) matrix. This format is highly efficient for row-wise operations, which aligns well with how recommendation algorithms, like Nearest Neighbors, process data.





In [60]:
from sklearn.neighbors import NearestNeighbors

from scipy.sparse import csr_matrix
book_sparse = csr_matrix(book_pivot)



model = NearestNeighbors(algorithm='brute')
model.fit(book_sparse)


NearestNeighbors(algorithm='brute')

## Generating Book Recommendations

After training our Nearest Neighbors model, we can now use it to find books similar to a specific book of interest. This is accomplished by querying the model with the ratings data of a single book and asking it to return a set of nearest neighbors; in other words, books that are most similar based on user ratings.

### Querying the Model

We selected a book from our dataset (at index 237 in the `book_pivot` DataFrame) to serve as our query point. The model then searches for the nearest neighbors to this book, considering the similarities in their ratings patterns:



In [62]:
distance, suggestion = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1), n_neighbors=6 )
print(distance)
print(suggestion)

[[ 0.         67.73129098 67.77802823 72.22091879 76.03909813 76.55027397]]
[[237 240 238 241 184 291]]


In [63]:
book_pivot.iloc[10,:]
book_title = book_pivot.index[10]


## Displaying Recommended Book Titles

After querying our Nearest Neighbors model for book recommendations, we obtain a list of indices representing the suggested books. These indices correspond to positions in our `book_pivot` DataFrame, which holds our data in a structured format suitable for our recommendation system.

### Iterating Over Suggestions

To reveal the actual titles of the recommended books, we iterate over the indices stored in the `suggestion` array. Since our query was focused on finding neighbors for a single book, `suggestion` contains a single row of indices, each pointing to a book in the `book_pivot` DataFrame considered similar to our query book:




In [64]:
# Assuming suggestion is a 2D array with one row of suggestions
for index in suggestion[0]:
    print(book_pivot.index[index])


Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Sorcerer's Stone (Book 1)
Exclusive
Jacob Have I Loved


## Working with Book Titles in the Pivot Table

The `book_pivot` DataFrame serves as the backbone of our recommendation system, organizing user ratings with books as rows and users as columns. To interact with this data structure effectively, especially for retrieving and working with book titles, we employ several Python operations.

### Accessing Book Titles

Book titles are stored as the index of the `book_pivot` DataFrame, allowing us to directly reference books by their position or by their title. For example:




In [67]:
book_pivot.index[3]


'4 Blondes'

In [69]:
# Storing all book titles in an array
book_names = book_pivot.index
print(book_names[2])  # Outputs: '2nd Chance'


2nd Chance


## Retrieving Image URLs for Recommended Books

After obtaining book recommendations, our next goal is to enrich the user experience by displaying the book covers. This involves retrieving the image URLs associated with each recommended book. The process leverages the titles of the recommended books to find their corresponding image URLs in the `final_rating` DataFrame, where such details are stored.

### Steps to Retrieve Image URLs

1. **Identify Book Titles**: We start with a list of book titles recommended by our model. These titles are extracted from the `book_pivot` DataFrame's index, based on the indices suggested by our Nearest Neighbors model.

2. **Find Corresponding Rows in Final Ratings DataFrame**: For each recommended book title, we locate the corresponding row in the `final_rating` DataFrame. This step is necessary because `final_rating` contains additional details about each book, including the image URLs.

3. **Extract Image URLs**: Once we have identified the rows corresponding to our recommended books in the `final_rating` DataFrame, we extract the `image_url` for each book.




In [79]:
import numpy as np

# Step 1: Extract recommended book titles from the suggestion array
# Initialize a list to store the titles of recommended books
book_titles = []
for book_id in suggestion[0]:  # Assuming suggestion[0] contains the indices of recommended books
    book_titles.append(book_pivot.index[book_id])

# Step 2: Find indices of recommended books in the final_rating DataFrame
# Initialize a list to hold indices of recommended books in final_rating
ids_index = []
for title in book_titles:
    # Find the first occurrence of each book title in final_rating and store its index
    ids = np.where(final_rating['Book-Title'] == title)[0][0]
    ids_index.append(ids)

# Step 3: Retrieve and print the image URLs for each recommended book
for idx in ids_index:
    # Extract the image URL for each book using its index
    url = final_rating.iloc[idx]['Image-URL-L']
    print(url)


http://images.amazon.com/images/P/0439064872.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0439136369.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0439139597.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/043936213X.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0446604232.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0064403688.01.LZZZZZZZ.jpg


## Book Recommendation Function: `recommend_book`

The `recommend_book` function is a core component of our book recommendation system, designed to provide users with suggestions for books similar to one they already like or are interested in. This function leverages our trained Nearest Neighbors model to find and recommend books based on user ratings patterns.

### How It Works

1. **Input**: The function accepts a single parameter, `book_name`, which is the title of the book for which recommendations are sought.
2. **Finding the Book**: It locates the given book in our `book_pivot` DataFrame, which contains the user ratings for books organized in a matrix form.
3. **Model Query**: Using the book's position in the matrix, the function queries the Nearest Neighbors model to find the closest matches based on the ratings data—essentially, books that users who liked the input book also enjoyed.
4. **Suggestions**: It then retrieves the titles of these suggested books, excluding the input book from the list to focus on providing new recommendations to the user.
5. **Output**: The suggested book titles are printed out, offering users a list of books they might enjoy next.

### Usage Example

To use the `recommend_book` function, simply call it with the title of a book as the argument:

```python
book_name = "Harry Potter and the Chamber of Secrets (Book 2)"
recommend_book(book_name)


In [80]:
def recommend_book(book_name):
    # Find the index of the book_name in the pivot table
    book_id = np.where(book_pivot.index == book_name)[0][0]
    # Use the model to find the 6 nearest neighbors (including the book itself)
    distances, suggestions = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1), n_neighbors=6)
    
    print(f"You searched for '{book_name}'.\n")
    print("The suggested books are:\n")
    
    # Iterate over the indices in the first (and only) row of suggestions
    for i in suggestions[0]:
        suggested_book_title = book_pivot.index[i]
        # Skip printing the searched book's title
        if suggested_book_title != book_name:
            print(suggested_book_title)


In [81]:

book_name = "Harry Potter and the Chamber of Secrets (Book 2)"
recommend_book(book_name)

You searched for 'Harry Potter and the Chamber of Secrets (Book 2)'.

The suggested books are:

Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Sorcerer's Stone (Book 1)
Exclusive
Jacob Have I Loved
